In [1]:
import json
import random
from subprocess import Popen, PIPE

In [2]:
# Datacenter's name: list of the brokers
hosts_dict = {0: [0, 1, 2], 1: [3, 4, 5], 2: [6, 7, 8]}

In [3]:
# List of the datacenters
hosts_dict[0]

[0, 1, 2]

In [4]:
default_json = """
{
    "partitions": [{
        "topic": "foo",
        "partition": 1,
        "replicas": [1, 2, 4]
    }],
    "version": 1
}
"""
print(default_json)


{
    "partitions": [{
        "topic": "foo",
        "partition": 1,
        "replicas": [1, 2, 4]
    }],
    "version": 1
}



In [5]:
parsed_string = json.loads(default_json)
desc_of_each_part = parsed_string['partitions'][0].copy()
parsed_string['partitions'].clear()

print(parsed_string)
print(desc_of_each_part)

{'version': 1, 'partitions': []}
{'replicas': [1, 2, 4], 'partition': 1, 'topic': 'foo'}


In [6]:
# Generate new list of the brokers (for storage partitions for each topics on the new datacenters)
def create_extended_broker_list():
    updated_broker_list = []

    for datacenter_num in hosts_dict:
        # List of the brokers in the current datacenter
        broker_list = hosts_dict[datacenter_num]
        current_broker = random.choice(broker_list)
        updated_broker_list.append(current_broker)

    return updated_broker_list

In [7]:
def create_json_for_partitions():
    # Getting a list of the topics from terminal
    p = Popen(['/home/temp/Projects/flink_metrics/kafka_2.10-0.10.2.1/bin/kafka-topics.sh --list --zookeeper localhost:2181'], shell=True, stdout=PIPE)

    # For each topic:
    for topic in p.stdout:
        # Write the name of the topic without \n
        desc_of_each_part['topic'] = (topic[:-1]).decode()
        # Determine brokers for storage partitions for this topic
        desc_of_each_part['replicas'] = create_extended_broker_list()
        # Add new info to json
        parsed_string['partitions'].append(desc_of_each_part.copy())
    return parsed_string

In [8]:
def main():
    list_for_json = create_json_for_partitions()
    result_json = json.dumps(list_for_json)
    
    # Magic happens here to make it pretty-printed
    pretty_json = json.dumps(json.loads(result_json), indent=4, sort_keys=True)
    print(pretty_json)
        
    # Now write output to a file
    json_file = open("partitions-to-move.json", "w")
    json_file.write(pretty_json)
    json_file.close()  


if __name__ == "__main__":
    main()

{
    "partitions": [
        {
            "partition": 1,
            "replicas": [
                1,
                3,
                7
            ],
            "topic": "__consumer_offsets"
        },
        {
            "partition": 1,
            "replicas": [
                1,
                4,
                6
            ],
            "topic": "fast-messages"
        },
        {
            "partition": 1,
            "replicas": [
                2,
                5,
                7
            ],
            "topic": "summary-markers"
        }
    ],
    "version": 1
}
